# Project 3: Where's Waldo?

Due Mar. 4th

So far, we've mostly focused on using imagery to do stuff for which it is better suited than a human: calculating camera locations from imagery, finding an optimal projective transform to stitch images together, and (soon) we'll be doing "structure from motion" in which we create 3D models of the world from collections of 2D images.  These are tasks primarily based around measuring things and doing calculations.  On the other side of the coin is object recognition (identifying the semantic content of a scene), and the best contemporary computer vision algorithms do object recognition at roughly the level of a 2 year old human (with some exceptions).  For this (mini-)project, we're going to delve into a topic that sort of straddles the line between these two general realms of computer vision.

As a motivating example, did you ever play the game Where's Waldo.  There are books filled with images like the following:
<img src='waldo_1.jpg'>
The objective, of course, is to find Waldo, the man in the red striped shirt and beanie wearing glasses.  He looks like this:
<img src='waldo_template.jpg'>
These scenes are (obviously) intended to have a bunch of visual clutter to make this task reasonably challenging.

Your task will be to come up with an algorithm that locates the template image (Waldo's face) and the target image (the larger scene).  This is called *template matching*, and it's a primitive form of feature recognition.  

## Implementation
### Template Matching
Template matching works in a way that is very similar to filtering:  slide the template image over every location in the target image, computing some sort of metric at each position.  In practice, one commonly used choice for an error metric is the one that you've already used for matching keypoint descriptors: z-normalized sum square error.  Another choice is [normalized cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation#Normalization).  Once these metrics have been computed, simply find the argmin (for SSE) or argmax (for NCC), and this will be the location of the best match.  

**Your task is to implement template matching.  Use 'waldo_template.jpg' as the template and 'waldo_1.jpg' as the target image.  Where's Waldo? **

### Not so fast!!!  What about scale!
Oh, no.  As it turns out, the template I've provided is not the same scale as the Waldo in the image.  To deal with this, you'll need to create an image pyramid for the template (See Szeliski 3.5, and [Mubarak Shah's lecture on this topic](https://www.youtube.com/watch?v=KO7jJt0WHag&feature=youtu.be)).  This essentially just means creating a sequence of downsampled images of the template, and trying each one in hopes that one of the resulting down-scaled templates matches the feature in the target image.  **Create a sequence of templates with which to perform feature matching, each one 1/2 the resolution of the previous (so 1/4 the total number of pixels).  To avoid aliasing, before downsampling perform a $\sigma=1$ Gaussian Blur of the image.  Once you've built your image pyramid, find the argmin/max in 3 dimensions (u,v,template scale)**.

## Generalization
**Waldo appears in every Where's Waldo image (obviously).  Try using the same technique on 'waldo_2.jpg'.  Does the algorithm work?**  I confess that I pulled the image of waldo for the template directly from 'waldo_1.jpg', so for the correct scale, there is something close to an exact match (i.e. SSE=0).  However, Waldo, while easily recognizable to the human eye after undergoing the small scale deformations associated with artistic license, is not so easily recognizable via template matching.  We will return to a similar problem when discussing object recognition, and hopefully this example will motivate the need to come up with representations of objects (like Waldo) that are more robust.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math as math
import skimage.io as img_io
import skimage.filters as filters
from skimage.transform import resize
from skimage.feature import match_template
from scipy.ndimage.filters import correlate
from sklearn.preprocessing import normalize

### INFO TO PLEASE TAKE INTO ACCOUNT ###
# I did this project solo. I went to class to try to meet my partners on wednesday (I got snowed in Friday and couldn't 
# make it to class at all, and they didn't show up. I don't know either of them, as well as not knowing their last names 
# so I couldn't email them, and with no moodle page I couldn't find them that way either, so I took on the project solo.
# To make up for that, I decided using libraries where possible would be the better route to actually having something that
# would function, as well as not having to reuse code without permission from former group mates who made the code. This is
# detailed slightly further below as well.


img = img_io.imread("waldo_1.jpg", as_gray=True)
template = img_io.imread("waldo_template.jpg", as_gray=True)

#Build pyramid by hand rather than using skimage gaussian_pyramid function, as well as label for output
#Used library functions for gaussian blur and resize as well for speed, and wasn't comfortable reusing code written by 
#previous group members since they did the convolution parts.
pyramid = []
pyramid.append(("Template",template))
for i in range(1,4):
    x = filters.gaussian(pyramid[i-1][1], sigma=1,multichannel=True)
    x = resize(x,(math.floor(x.shape[0]/2),math.floor(x.shape[1]/2)))
    pyramid.append(("Downscale #" + str(i),x))

#Show image and show each step of the pyramid
img_io.imshow(img)
plt.show()
for item in pyramid:
    print(item[0]+":")
    img_io.imshow(item[1])
    plt.show()



#Using skimage template match since it is well implemented. Mine took a VERY long time, left as comment to show what I 
#attempted But used the library function for speed and corretness. The library function uses a 
#normalized cross-correlation so I needed to use argmax to get the guessed location. I generally code with the 
#philosophy generally knowing what it is doing is the goal, so it is better to use libraries and not reinvent the wheel 
#as long as you know the process, rather than having to know the specifics at all times.
outputs = []
for item in pyramid:
    cmp = item[1]
    guess = match_template(img, cmp)
    ij = np.unravel_index(np.argmax(guess), guess.shape)
    x, y = ij[::-1]
    outputs.append((item[0],x,y))

print("Guesses per comparison in x,y format from skimage correlation:")
for item in outputs:
    print(item[0]," Guess:",item[1],",",item[2])
    
print("\n Starting second correlation \n")
#This is testing with scipy's correlate function instead and using sklearn to normalize to see if they are similar.
outputs2 = []
for item in pyramid:
    cmp = item[1]
    img = normalize(img)
    cmp = normalize(cmp)
    guess = correlate(img, cmp)
    ij = np.unravel_index(np.argmax(guess), guess.shape)
    x, y = ij[::-1]
    outputs2.append((item[0],x,y))
    

    
print("Guesses per comparison in x,y format from scipy correlation:")
for item in outputs2:
    print(item[0]," Guess:",item[1],",",item[2])

#Based on manually checking with several pyramids, looks like after being downscaled twice it finds Waldo (Downscale #2) from
#the skimage template matching, but the scipy correlation isn't as accurate, while also taking significantly longer


#Image dimensions
# img_x = img.shape[0]
# img_y = img.shape[1]

# My attempt, extremely slow and not 100% sure if right
# for item in pyramid:
#     x_dim = item.shape[0]
#     y_dim = item.shape[1]
#     out = np.zeros((img_y - y_dim,img_x - x_dim))
#     for i in range(img_y - y_dim):
#         for j in range(img_x - x_dim):
#             sum = 0.0
#             for x in range(x_dim):
#                 for y in range(y_dim):
#                     sum += (img[i + x][j + y] - item[x][y])
#             out[i][j] = sum
#     print("Template Done")
#     min_ind = np.unravel_index(np.argmin(out, axis=None), out.shape)
#     print(min_ind)
            

